In [9]:
from pymongo import MongoClient
import mysql.connector
from datetime import datetime
import pandas as pd

# MongoDB setup
mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['Coinut']
collection = mongo_db['transaction']
# Fetch all doc data
data1 = collection.find()

# MySQL setup
mysql_db = mysql.connector.connect(
  user='root', password='liuwinq1', host='localhost', database='coinut')
mysql_cursor = mysql_db.cursor()

In [6]:
# First, let's drop the table if it already exists
mysql_cursor.execute("DROP TABLE IF EXISTS Transactions")


# let's create the MySQL table
mysql_cursor.execute("""
    CREATE TABLE Transactions (
        transaction_id VARCHAR(255) PRIMARY KEY,
        instId INT,
        fill_price FLOAT,
        fill_qty FLOAT,
        id BIGINT,
        makerComm_amount FLOAT,
        makerComm_currency INT,
        matchedOrder_clientOrdId BIGINT,
        order_clientOrdId BIGINT,
        takerComm_amount FLOAT,
        takerComm_currency INT,
        timestamp Date,
        trans_id BIGINT,
        type VARCHAR(255),
        transaction_value_sgd FLOAT
    )
""")

In [7]:
# Now, we iterate over the MongoDB data and insert it into MySQL
for document in data1:
    transaction_id = str(document['_id'])  # Convert ObjectId to string
    instId=int(document['order']['instId'])
    fill_price = float(document['fillPrice'])
    fill_qty = float(document['fillQty'])
    id = int(document['id'])
    makerComm_amount = float(document['makerComm']['amount'])
    makerComm_currency = int(document['makerComm']['currency'])
    matchedOrder_clientOrdId = int(document['matchedOrder']['clientOrdId'])
    order_clientOrdId = int(document['order']['clientOrdId'])
    takerComm_amount = float(document['takerComm']['amount'])
    takerComm_currency = int(document['takerComm']['currency'])
    
     # Convert timestamp to datetime format
    timestamp = document['timestamp']
    dt = datetime.utcfromtimestamp(int(timestamp) / 1e6).strftime('%Y-%m-%d %H:%M:%S.%f')
    
    timestamp = dt
    trans_id = int(document['transId'])
    type = document['type']
    transaction_value_sgd=float(document['transaction_value_sgd'])

    # Construct the INSERT query
    query = """
        INSERT INTO Transactions (
            transaction_id, instId, fill_price, fill_qty, id, makerComm_amount, 
            makerComm_currency, matchedOrder_clientOrdId, order_clientOrdId,
            takerComm_amount, takerComm_currency, timestamp,
            trans_id, type, transaction_value_sgd
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)
    """

    # Execute the query
    mysql_cursor.execute(query, (transaction_id, instId, fill_price, fill_qty, id, makerComm_amount,
                                 makerComm_currency, matchedOrder_clientOrdId, order_clientOrdId,
                                 takerComm_amount, takerComm_currency, timestamp,
                                 trans_id, type,transaction_value_sgd))

# Commit the changes
mysql_db.commit()

# Close the MySQL cursor and connection
mysql_cursor.close()
mysql_db.close()

In [17]:
# First, let's drop the table if it already exists
mysql_cursor.execute("DROP TABLE IF EXISTS CryptoPairs")

# Now, let's create the MySQL table
mysql_cursor.execute("""
    CREATE TABLE CryptoPairs (
        instId INT PRIMARY KEY,
        pair_name VARCHAR(255)
    )
""")

# Crypto_pair Table

In [18]:
# Dictionary
crypto_pairs = {1: 'LTC, BTC', 2: 'ETH, BTC', 3: 'ETC, BTC', 490569: 'ETH, LTC', 490588: 'ETC, LTC', 490589: 'BTC, USDT', 490590: 'LTC, USDT', 490591: 'ETH, USDT', 490592: 'ETC, USDT', 852382: 'XMR, USDT', 852383: 'XMR, BTC', 852384: 'XMR, LTC', 1111976: 'ZEC, USDT', 1111977: 'ZEC, BTC', 1111978: 'ZEC, LTC', 1801843: 'BTC, CAD', 1801844: 'LTC, CAD', 1801845: 'ETH, CAD', 1885998: 'ZEC, CAD', 2372505: 'BTC, XSGD', 2372506: 'LTC, XSGD', 2372507: 'ETH, XSGD', 2372508: 'USDT, XSGD', 2372510: 'DAI, XSGD', 2372511: 'ETC, XSGD', 2372512: 'BTC, XIDR', 852220: 'USDT, USD', 852234: 'LTC, USD', 852379: 'BTC, USD', 852380: 'BTC, SGD', 852381: 'LTC, SGD', 899746: 'USDT, SGD', 1063804: 'ETH, SGD', 1885996: 'ZEC, USD', 1885999: 'ETH, USD', 2372503: 'ETC, SGD', 2372504: 'DAI, SGD', 2372509: 'XSGD, SGD'}
# Now, iterate over the dictionary and insert its elements into MySQL
for instId, pair_name in crypto_pairs.items():
    query = """
        INSERT INTO CryptoPairs (instId, pair_name) 
        VALUES (%s, %s)
    """
    mysql_cursor.execute(query, (instId, pair_name))

# Commit the changes
mysql_db.commit()

# Close the MySQL cursor and connection
mysql_cursor.close()
mysql_db.close()

# Currency Table

In [12]:
# First, let's drop the table if it already exists
mysql_cursor.execute("DROP TABLE IF EXISTS currency")

# Now, let's create the MySQL table
mysql_cursor.execute("""
    CREATE TABLE currency (
        currency_id INT PRIMARY KEY,
        currency_name VARCHAR(255)
    )
""")

In [13]:
# Dictionary
currency_index = {
    0: 'BTC', 1: 'USD', 2: 'LTC', 3: 'ETH', 4: 'ETC', 5: 'USDT', 6: 'XMR', 7: 'BCH', 8: 'BTG', 9: 'B2X', 
    10: 'SBTC', 11: 'SGD', 12: 'CAD', 13: 'EUR', 14: 'NTD', 15: 'JPY', 16: 'GBP', 17: 'LCH', 18: 'CHF', 
    19: 'ZEC', 20: 'ADA', 21: 'MYR', 22: 'DAI', 23: 'XSGD', 24: 'XIDR'}# Now, iterate over the dictionary and insert its elements into MySQL

for currency_id, currency_name in currency_index.items():
    query = """
        INSERT INTO currency (currency_id, currency_name) 
        VALUES (%s, %s)
    """
    mysql_cursor.execute(query, (currency_id, currency_name))

# Commit the changes
mysql_db.commit()

# Close the MySQL cursor and connection
mysql_cursor.close()
mysql_db.close()

In [ ]:
cnx = mysql.connector.connect(
  user='root', password='liuwinq1', host='localhost', database='coinut')

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# create sqlalchemy engine
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}".format(
    user="root",
    password="liuwinq1",
    host="localhost",
    dbname="coinut"
))

# replace 'username', 'password', 'hostname', 'database_name' with your actual credentials

# read the data from the `price` table
df = pd.read_sql("SELECT * FROM price", engine)

# convert `time` to datetime and extract date
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date

# keep only the first row for each date
df = df.groupby('date').first().reset_index()

# write the DataFrame back to a new table in your MySQL database
df.to_sql('price_transformed', engine, if_exists='replace', index=False)

In [ ]:
from datetime import datetime
timestamp = 1640995314539072 
dt = datetime.utcfromtimestamp(int(1640995314539072 ) / 1e6).strftime('%Y-%m-%d')
print(dt)

# Commision value

In [10]:
# Load the data from the tables into pandas DataFrames
transactions_df = pd.read_sql('SELECT * FROM Transactions', mysql_db)
currency_df = pd.read_sql('SELECT * FROM currency', mysql_db)
price_transformed_df = pd.read_sql('SELECT * FROM price_transformed', mysql_db)

# Map currency_id to currency_name in the transactions DataFrame
currency_id_to_name = currency_df.set_index('currency_id')['currency_name'].to_dict()
transactions_df['makerComm_currency_name'] = transactions_df['makerComm_currency'].map(currency_id_to_name)
transactions_df['takerComm_currency_name'] = transactions_df['takerComm_currency'].map(currency_id_to_name)

# Initialize new columns for makerComm_value and takerComm_value
transactions_df['makerComm_value'] = 0.0
transactions_df['takerComm_value'] = 0.0

# Calculate makerComm_value and takerComm_value for each transaction
for idx, transaction in transactions_df.iterrows():
    maker_price = price_transformed_df.loc[price_transformed_df['date'] == transaction['timestamp'], transaction['makerComm_currency_name']].values[0]
    taker_price = price_transformed_df.loc[price_transformed_df['date'] == transaction['timestamp'], transaction['takerComm_currency_name']].values[0]
    transactions_df.loc[idx, 'makerComm_value'] = transaction['makerComm_amount'] * maker_price
    transactions_df.loc[idx, 'takerComm_value'] = transaction['takerComm_amount'] * taker_price


C:\anaconda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\anaconda\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
import mysql.connector

# Initialize MySQL cursor
mysql_cursor = mysql_db.cursor()

# Function to check if a column exists in a table
def column_exists(table, column):
    mysql_cursor.execute(f"SHOW COLUMNS FROM {table} LIKE '{column}'")
    result = mysql_cursor.fetchone()
    if result:
        return True
    return False

# Create new columns in the Transactions table if they do not exist
if not column_exists('Transactions', 'makerComm_value'):
    mysql_cursor.execute("ALTER TABLE Transactions ADD COLUMN makerComm_value FLOAT")
else:
    print("Column 'makerComm_value' already exists.")
    
if not column_exists('Transactions', 'takerComm_value'):
    mysql_cursor.execute("ALTER TABLE Transactions ADD COLUMN takerComm_value FLOAT")
else:
    print("Column 'takerComm_value' already exists.")
    
# Commit the changes
mysql_db.commit()

# Iterate through each row in the dataframe
# Prepare a MySQL UPDATE statement to update the makerComm_value and takerComm_value columns
query = """
    UPDATE Transactions SET makerComm_value = %s, takerComm_value = %s WHERE transaction_id = %s
"""

for idx, row in transactions_df.iterrows():
    # Prepare the data for the UPDATE statement
    data = (row['makerComm_value'], row['takerComm_value'], row['transaction_id'])
    # Execute the UPDATE statement
    mysql_cursor.execute(query, data)

# Commit the changes
mysql_db.commit()

# Close the MySQL cursor and connection
mysql_cursor.close()
mysql_db.close()


Column 'makerComm_value' already exists.
Column 'takerComm_value' already exists.
